In [1]:
# generic functions
# some generic constants and arrays are defined at the beginning of this file
from modules.my_functions import *

# functions for the dark matter halo catalogue
from modules.dark_matter_haloes import *

# functions for assigning and evolving galaxies
from modules.galaxies import *

# functions for building black holes
from modules.black_holes import *

from modules.kuv_correction_imf_Lapi import *

# function for the Mancuso+2016 luminosity function
#from Mancuso_Lapi_SFRF.SFRF import *

from colossus.halo import mass_so
from colossus.halo import concentration
from colossus.halo import profile_nfw
from colossus.lss import mass_function
from colossus.halo import mass_adv

from tqdm import tqdm
import time
import os


In [2]:
input_params = read_input_params("input/explanatory.txt")

Reading parameters from file:
input/explanatory.txt



In [3]:
nhalo, masses, orders, mass_accs, loghalo_masses, mhlog_arr, dmhdtlog_arr = load_SatGen_halo_catalogue_highz(input_params, mhlog_Min=12., mhlog_Max=15.5)

Total number of haloes: 153235
Minimum halo mass in file: 11.000000654265582


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.81s/it]


Initial number of haloes: 153235
Reduced number of haloes 77


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77/77 [00:04<00:00, 16.29it/s]


In [4]:
tree=unpack_SatGen_merger_tree_highz(z_grid,nz_grid,nhalo,masses,mass_accs,orders,mhlog_arr, mhdotlog, z_SatGen)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 2696.92it/s]


Dynamical friction timescales computed.
Mass ratios computed.
Fudge factor computed.
Merging timescales computed
Redshift at merging computed.


In [5]:
# halo mass function
hmf, hmflog, hmf_cum, norm, volume, cube_side, hmf_cat = halo_mass_function(input_params.z0,z_grid,nz_grid,nhalo,mhlog_arr,mhlog,nmh,mhlog_min)

Volume cube side: 32.788806 Mpc


In [6]:
mhlog_crit = halo_mass_crit_quench_givenparams(1.4, 1., input_params.logMhDekkel, input_params.halo_quenching)
red_har = np.arange(0,11.1,0.25)
phihar = [ np.loadtxt("../../SatGen/Daniel/HARfunctions/HAR_function_haloquench_z_%.2f.txt"%Z) for Z in red_har ]
dNdVdlogmhdot_active = load_SatGen_halo_acc_rate_function(input_params.halo_quenching, input_params.mergers_quenching, mhdotlog, nz, nmh, z)

In [7]:
phisfrLF=sfr_function(z,nz,sfrlog,nsfr)

In [8]:
sfr_am_, sfrlog_am_ = abundance_matching_LB(z,nz,mhdotlog,nmhdot, phisfrLF,sfrlog,nsfr, dNdVdlogmhdot_active, input_params.sigma_sfr, input_params.delay)
sfrlog_am = np.zeros((mhdotlog.size, nz_grid))
for imhdot in range(mhdotlog.size):
    sfrlog_am[imhdot,:] = interp1d(z,sfrlog_am_[imhdot,:],fill_value="extrapolate")(z_grid)
sfr_am = 10.**sfrlog_am


In [9]:
z_start=12.

sfrlog_cat, sfrlog_cat_real = sfr_catalogue(z_start, lt_grid, z_grid,nz_grid,nhalo,sfrlog_am, input_params.sigma_sfr,mhdotlog,dmhdtlog_arr,mhlog_arr, input_params.sfr_delay_alpha, mhlog_crit, input_params.scatter_mhlog_crit, 1, input_params.include_SNfeedback, input_params.logMh_SNfeedback, input_params.delay)
sfrlog_cat[:,z_grid>z_start]=-65. # impongo -65 perché voglio che la SFR sia 0, ma non deve essere contata come quenchata (le quenchate vengono classificate come logsfr==-66)

for iz in range(nz_grid):
    sfrlog_cat[np.logical_not(np.isfinite(sfrlog_cat[:,iz])),iz]=-65.

for ihalo in range(nhalo):
    sfrlog_cat_real[ihalo,z_grid>z_start]=-66.

print("SFRs assigned")

mstar_integrated, logmstar_integrated = integrate_accretion_rates_across_time(z_grid, sfrlog_cat_real)
print("integration finished")


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 31765.65it/s]

SFRs assigned
integration finished


In [11]:
mstar_integrated_with_mergers, logmstar_integrated_with_mergers, tree = add_mergers_SatGen_tree_highz(z_grid,nz_grid,nhalo,mhlog_arr,logmstar_integrated,mstar_integrated,tree)


Stellar masses computed
